# Expected Completion

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist #function to create adjacency matrix

In [2]:
plays = pd.read_csv("data/pass_receiver.csv")
pff = pd.read_csv("data/pffScoutingData.csv")


read in weekly tracking files and concatenate into one tracking file, then delete weeklies

In [3]:
week1 = pd.read_csv("data/week1.csv")
week2 = pd.read_csv("data/week2.csv")
week3 = pd.read_csv("data/week3.csv")
week4 = pd.read_csv("data/week4.csv")
week5 = pd.read_csv("data/week5.csv")
week6 = pd.read_csv("data/week6.csv")
week7 = pd.read_csv("data/week7.csv")
week8 = pd.read_csv("data/week8.csv")
tracking = pd.concat([week1,week2,week3,week4,week5,week6,week7,week8])


In [4]:
#save memory
del week1,week2,week3,week4,week5,week6,week7,week8

In [5]:
tracking.columns

Index(['gameId', 'playId', 'nflId', 'frameId', 'time', 'jerseyNumber', 'team',
       'playDirection', 'x', 'y', 's', 'a', 'dis', 'o', 'dir', 'event'],
      dtype='object')

add unique identifier of each play and frame (some playIds repeat over games)

In [6]:
tracking["comb_id"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")
pff["comb_id"] = pff["gameId"].astype("str")+" "+pff["playId"].astype("str")
plays["comb_id"] = plays["gameId"].astype("str")+" "+plays["playId"].astype("str")
tracking["comb_and_frame"] = tracking["gameId"].astype("str")+" "+tracking["playId"].astype("str")+" "+tracking["frameId"].astype("str")

isolate frames at time of pass

In [7]:
#isolate just forward passes and leave out a certain frame (duplicate forward pass for a play)
just_forward_passes = tracking[(tracking["event"]=="pass_forward") & (tracking["comb_and_frame"]!="2021091203 3740 32")& (tracking["team"]!="football")]
#bring in receiver info from plays
tracking_plays =just_forward_passes.merge(plays[["comb_id","possessionTeam","is_pass_failed","PassReceiver_nflId"]], how="left",on="comb_id" )
#and qb info from pff
tracking_merged_all = tracking_plays.merge(pff[["comb_id","nflId","pff_role"]], how = "left",on=["comb_id","nflId"])

In [8]:
len(just_forward_passes)

166034

In [9]:
len(tracking_plays)

166276

isolate tracking to just QB, OL, receiver and defense

In [10]:
tracking_players_filtered = tracking_merged_all[(tracking_merged_all["nflId"]==tracking_merged_all["PassReceiver_nflId"])|(tracking_merged_all["pff_role"]=="Pass")|(tracking_merged_all["pff_role"]=="Pass Block")|(tracking_merged_all["team"]!=tracking_merged_all["possessionTeam"])]

# Function to take in 1 pass play and put it in graph format( list containing adjacency matrix, features, y) this will then be parallelized for all passes in the data

In [11]:
sample = tracking_players_filtered[tracking_players_filtered.comb_id == "2021090900 97"]
sample_dist = sample[["x","y"]]

In [57]:

adj = cdist(sample_dist.values, sample_dist.values)
sample = sample.reset_index()
for index,row in sample.iterrows():
    if (row["pff_role"]=="Coverage" )| (row["pff_role"]=="Pass Route"):
        condition = ((sample["pff_role"]=="Pass Block")|(sample["pff_role"]=="Pass")|(sample["pff_role"]=="Pass Rush"))
    else:
        condition = ((sample["pff_role"]=="Coverage")|(sample["pff_role"]=="Pass Route"))
    adj[index][condition] = np.NaN
               
    

if role = Pass Route or coverage clear all not that, if role = QB or pass block or pass rush, just keep them

In [47]:
adj

array([[ 0.        ,  3.73507697,  3.59777709,  3.78842975,  4.32786321,
         9.98721683, 27.34748983, 31.79944811, 21.00506844, 21.88118141,
        11.32525055, 37.95209876,  6.27459959, 14.08077413,  3.32085832,
        18.41671252,  2.45079579,  1.78507703],
       [ 3.73507697,  0.        ,  4.0718055 ,  3.59680414,  3.61247837,
         7.16497732, 24.68029173, 28.63740386, 17.3403835 , 18.17851754,
         8.41080852, 34.21754228,  4.73305398, 10.64805146,  2.83557754,
        14.68885632,  2.84429253,  2.32845442],
       [ 3.59777709,  4.0718055 ,  0.        ,  0.69065187,  1.25251746,
         7.29414148, 24.15944536, 29.01162698, 20.5315976 , 21.19327488,
         8.63229402, 36.66754696,  3.09027507, 14.37640428,  1.26051577,
        17.16228714,  1.4707821 ,  4.17900706],
       [ 3.78842975,  3.59680414,  0.69065187,  0.        ,  0.62625873,
         6.69845505, 23.71833468, 28.46904459, 19.8482367 , 20.50484089,
         8.04328913, 35.9829404 ,  2.58642611, 13.741

In [ ]:
def one_play_graph_function(data):
    one_play_formatted_list = []
    #first put play in adjacency matrix format, the distance of all players to each other (this will then be filtered to just link relevant players to QB or WR)

    #then put play in features format, getting a table containing position, speed, orientation etc

    #then get y, the response variable for this play (binary pass completed or not)
    return one_play_formatted_list